In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter, defaultdict
import pylab as pl
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

In [2]:
data = pd.read_excel("C:\Users\Niku\Documents\ML Project\data2.5k.xlsx")
print data.shape
data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['DateOnly'] = data['Date'].dt.day
print data.shape

(2500, 13)
(2500, 16)


In [3]:
#X = data[['DayOfWeek', 'PdDistrict', 'X', 'Y', 'Year', 'Month', 'DateOnly', 'Time']] #Acc = 0.15
#X = data[['DayOfWeek', 'PdDistrict', 'X', 'Y', 'Year', 'Month', 'DateOnly']] #Acc = 0.16
#X = data[['DayOfWeek',  'X', 'Y', 'Year', 'Month', 'DateOnly', 'Time']] #Acc = 0.155
#X = data[[ 'PdDistrict', 'X', 'Y', 'Year', 'Month', 'DateOnly', 'Time']] #Acc = 0.17 with test size 0.05
#X = data[['DayOfWeek', 'PdDistrict', 'X', 'Y',  'Time']] #Acc = 0.15
X = data[['DayOfWeek', 'PdDistrict']]
print X.shape
X = X.values
y = data['Category']
y = y.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5, random_state=1)
print X_train.shape
print X_test.shape
print y_test.shape
print type(y_train)

(2500, 2)
(1250L, 2L)
(1250L, 2L)
(1250L,)
<type 'numpy.ndarray'>


In [4]:
labels = np.unique(y_train)
#No Smoothing
def Prob(y_vector):
    total = len(y_vector)
    r_dict = {}
    cnt = Counter(y_vector)
    for key, value in cnt.items():
        r_dict[key] = float(value)/total
    return r_dict

#Smoothing 
#def Prob(y_vector):
#    total = len(y_vector)
#    r_dict = {}
#    cnt = Counter(y_vector)
#    l = len(cnt.items())
#    for key, value in cnt.items():
#        r_dict[key] = (float(value)+1)/(total + l)
#    return r_dict

dic = {}  
#ClassifyingByLabels 
def ClassifyByLabels(X_matrix, y_vector):
    
    [rows, columns] = np.shape(X_matrix)
    total = rows
    for label in labels:
        dic[label] = defaultdict(list)
    for label in labels:
        row_indices = np.where(y_vector == label)[0] 
        X_current = X_train[row_indices, :]
        for i in range(0,columns):
            dic[label][i] += list(X_current[:,i])
    for label in labels:
        for col in range(0, columns):
            dic[label][col] = Prob(dic[label][col])
            #print dic[label][col]
            
ClassifyByLabels(X_train, y_train)   
crime_dict = Prob(y_train)
crime_dict
#crime_dict[0:5]

{u'ARSON': 0.0024,
 u'ASSAULT': 0.1032,
 u'BAD CHECKS': 0.0008,
 u'BRIBERY': 0.0016,
 u'BURGLARY': 0.032,
 u'DISORDERLY CONDUCT': 0.0072,
 u'DRIVING UNDER THE INFLUENCE': 0.0032,
 u'DRUG/NARCOTIC': 0.068,
 u'DRUNKENNESS': 0.0064,
 u'EMBEZZLEMENT': 0.0024,
 u'FORGERY/COUNTERFEITING': 0.0128,
 u'FRAUD': 0.0272,
 u'GAMBLING': 0.0008,
 u'KIDNAPPING': 0.0024,
 u'LARCENY/THEFT': 0.1592,
 u'MISSING PERSON': 0.0256,
 u'NON-CRIMINAL': 0.0808,
 u'OTHER OFFENSES': 0.1624,
 u'PROSTITUTION': 0.0136,
 u'ROBBERY': 0.024,
 u'RUNAWAY': 0.0024,
 u'SECONDARY CODES': 0.0072,
 u'SEX OFFENSES, FORCIBLE': 0.0064,
 u'STOLEN PROPERTY': 0.0024,
 u'SUSPICIOUS OCC': 0.0312,
 u'TRESPASS': 0.008,
 u'VANDALISM': 0.0424,
 u'VEHICLE THEFT': 0.084,
 u'WARRANTS': 0.0712,
 u'WEAPON LAWS': 0.0088}

In [5]:
datafr = pd.DataFrame(columns=labels)
#Output maximum probability category
def maximum(dict1):
    values = dict1.values()
    keys = dict1.keys()
    max_index = values.index(max(values))
    max_key = keys[max_index]
    return max_key

#Testing on X_test

def Prediction(elem, ids):
    predict_dict = {}
    for label in labels:
        prior = crime_dict[label]
        likelihood = prior
        for i in range(0, len(elem)):
            if elem[i] in dic[label][i].keys():
                likelihood *= dic[label][i][elem[i]]
            else:
                likelihood *= 0.0
        predict_dict[label] = likelihood
    datafr.loc[ids] = predict_dict
    return maximum(predict_dict)   
        

def classify(X_matrix):
        predicted_y = []
        no_rows, no_cols = np.shape(X_matrix)
        for i in range(0,no_rows):
            elem = X_matrix[i,:]
            prediction = Prediction(elem, i)
            predicted_y.append(prediction)
        return predicted_y

classify(X_test)
predicted_y = classify(X_test)
y_labels = np.unique(y_test)
y_pred = np.array(predicted_y)
#for y_label in y_labels:
    #f1 = f1_score(y_pred, y_test, y_label, average="weighted")
    #print("F1-score on the test-set for class %s is: %s" % (y_label, f1))

In [6]:
X_test

array([[u'Tuesday', u'TENDERLOIN'],
       [u'Wednesday', u'TENDERLOIN'],
       [u'Monday', u'TENDERLOIN'],
       ..., 
       [u'Wednesday', u'BAYVIEW'],
       [u'Wednesday', u'TARAVAL'],
       [u'Friday', u'CENTRAL']], dtype=object)

In [7]:
print y_test.shape
print y_pred.shape
accuracy_score(y_test, y_pred)

(1250L,)
(1250L,)


0.16

In [8]:
from sklearn.metrics import log_loss
log_loss(y_test, datafr, labels = labels)

4.0141446595277577

In [9]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

                             precision    recall  f1-score   support

                      ARSON       0.00      0.00      0.00         2
                    ASSAULT       0.12      0.06      0.08       102
                 BAD CHECKS       0.00      0.00      0.00         4
                   BURGLARY       0.25      0.03      0.06        62
         DISORDERLY CONDUCT       0.00      0.00      0.00         3
DRIVING UNDER THE INFLUENCE       0.00      0.00      0.00         4
              DRUG/NARCOTIC       0.20      0.18      0.19       115
                DRUNKENNESS       0.00      0.00      0.00         5
               EMBEZZLEMENT       0.00      0.00      0.00         4
     FORGERY/COUNTERFEITING       0.00      0.00      0.00        10
                      FRAUD       0.00      0.00      0.00        23
                 KIDNAPPING       0.00      0.00      0.00         2
              LARCENY/THEFT       0.19      0.35      0.25       175
                LIQUOR LAWS      

C:\ProgramData\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [10]:
#font_scale = 3.5
#cm = confusion_matrix(y_test, y_pred, labels)
#fig = plt.figure(figsize=(55,55))
#ax = fig.add_subplot(111)
#cax = ax.matshow(cm)
#pl.title('Confusion matrix of the classifier')
#fig.colorbar(cax)
#ax.set_xticklabels(labels, rotation=90)
#ax.set_yticklabels(labels)
#pl.xlabel('Predicted')
#pl.ylabel('True')
#pl.show()

In [11]:
#y_test_series = pd.Series(y_test, name = 'Actual')
#y_pred_series = pd.Series(y_pred, name = "Predicted")
#df_confusion = pd.crosstab(y_test_series, y_pred_series, margins=True)
#df_conf_norm = df_confusion / df_confusion.sum(axis=1)

#writer = pd.ExcelWriter('PythonExport.xlsx')
#df_confusion.to_excel(writer,'Sheet1')
#writer.save()

In [12]:
#def plot_confusion_matrix(df_confusion, title='Confusion matrix', cmap=plt.cm.gray_r):
#    plt.matshow(df_confusion, cmap=cmap) # imshow
    #plt.title(title)
#    plt.colorbar()
#    tick_marks = np.arange(len(df_confusion.columns))
#    plt.xticks(tick_marks, df_confusion.columns, rotation=45)
#    plt.yticks(tick_marks, df_confusion.index)
    #plt.tight_layout()
#    plt.ylabel(df_confusion.index.name)
#    plt.xlabel(df_confusion.columns.name)
    
#plot_confusion_matrix(df_confusion)

In [13]:
datafr

,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,...,RUNAWAY,SECONDARY CODES,"SEX OFFENSES, FORCIBLE",STOLEN PROPERTY,SUSPICIOUS OCC,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,0.000000,0.000726,0.0,0.0,0.00016,0.000000,0.0000,0.000988,0.0000,0.000000,...,0.000000,0.000000,0.0000,0.000000,0.000185,0.00000,0.000030,0.000030,0.000503,0.000000
1,0.000000,0.002288,0.0,0.0,0.00072,0.000000,0.0000,0.001129,0.0006,0.000000,...,0.000000,0.000000,0.0000,0.000000,0.000738,0.00000,0.000151,0.000122,0.001007,0.000145
2,0.000000,0.000558,0.0,0.0,0.00016,0.000000,0.0000,0.000847,0.0000,0.000000,...,0.000000,0.000000,0.0000,0.000000,0.000185,0.00000,0.000030,0.000091,0.000575,0.000073
3,0.000000,0.000651,0.0,0.0,0.00030,0.000000,0.0000,0.000753,0.0003,0.000000,...,0.000000,0.000000,0.0000,0.000000,0.000246,0.00008,0.000362,0.001234,0.000467,0.000000
4,0.000000,0.000558,0.0,0.0,0.00040,0.000000,0.0000,0.002541,0.0000,0.000000,...,0.000000,0.000000,0.0000,0.000000,0.000308,0.00000,0.000211,0.000160,0.001294,0.000218
5,0.000000,0.000521,0.0,0.0,0.00156,0.000089,0.0000,0.000160,0.0002,0.000267,...,0.000000,0.000000,0.0000,0.000000,0.000164,0.00024,0.000423,0.001493,0.000171,0.000000
6,0.000000,0.000912,0.0,0.0,0.00078,0.000000,0.0000,0.000800,0.0006,0.000000,...,0.000000,0.000000,0.0000,0.000000,0.000246,0.00024,0.000423,0.000960,0.000683,0.000000
7,0.000000,0.002344,0.0,0.0,0.00104,0.000356,0.0004,0.002240,0.0002,0.000000,...,0.000000,0.000000,0.0000,0.000000,0.000492,0.00072,0.001057,0.000640,0.003587,0.000218
8,0.000000,0.001054,0.0,0.0,0.00008,0.000000,0.0000,0.000508,0.0000,0.000000,...,0.000000,0.000000,0.0000,0.000000,0.000308,0.00016,0.000151,0.001006,0.000575,0.000145
9,0.000000,0.001885,0.0,0.0,0.00024,0.000000,0.0000,0.000489,0.0002,0.000000,...,0.000000,0.000711,0.0000,0.000000,0.001149,0.00016,0.000543,0.002590,0.000449,0.000000
